In [1]:
!tar -zxvf dataset_roboflow.tar.gz && tar -zxvf dataset_sketch_it.tar.gz

dataset_roboflow/train/00021_png.rf.f33255cfe99ce33a5b61c360f67a8c58.jpg
dataset_roboflow/train/image_38_png.rf.e69ffcc3c101dd4cf1a1f50d8389da52.jpg
dataset_roboflow/valid/image_74_png.rf.52beed189736dbe45515e7820607e606.jpg
dataset_roboflow/new_train/00034_png.rf.cdaed64c2e2f8fe839439f6c387436e5.jpg
dataset_roboflow/new_train/image_161_png.rf.cfd20b4fe1695e91f26646da22150381.jpg
dataset_roboflow/new_train/image_97_png.rf.82f9aba938949bedc9318178f2d42292.jpg
dataset_roboflow/train/image_36_png.rf.cf8a6d8e41936d90a3ea6d59052cc4e5.jpg
dataset_roboflow/new_train/image_91_png.rf.6153c5683671b88991c985917573bc58.jpg
dataset_roboflow/train/image_182_png.rf.6a6b8609bedf8aedb3fb4208b3231579.jpg
dataset_roboflow/new_train/image_108_png.rf.81a586ab7642048a8fe556320feb4af2.jpg
dataset_roboflow/train/image_55_png.rf.3f895c00b966af76d1057ee8f2218ace.jpg
dataset_roboflow/train/image_107_png.rf.f48cbcdeaf88cb6aad0a97358d70e81c.jpg
dataset_roboflow/train/image_200_png.rf.35c299086ff7729f94fae9568a5748

dataset_roboflow/train/image_49_png.rf.f630a08315a3e8d6bffaf50d483711c9.jpg
dataset_roboflow/new_train/image_31_png.rf.ad3c348be7130f741b2d841f3a7ac0d8.jpg
dataset_roboflow/new_train/image_117_png.rf.26c32f75a554a987d319c2d8e3ce09cd.jpg
dataset_roboflow/new_train/image_3_png.rf.cc2c88eced22cc3d8d6aedf240344011.jpg
dataset_roboflow/valid/00033_png.rf.059190b554c8273dae6620c31a6788c3.jpg
dataset_roboflow/train/image_157_png.rf.25f4ca2d300bf8ba063bd65a0184d341.jpg
dataset_roboflow/new_train/00018_png.rf.ca1c46d1806bcfdb955d0ec2f5ebc93a.jpg
dataset_roboflow/valid/image_198_png.rf.7bf38b73e026959c122edf31c4aee90e.jpg
dataset_roboflow/train/image_139_png.rf.80130f9de11a105ad496dffaf2101015.jpg
dataset_roboflow/new_train/image_113_png.rf.2e8877153058922337bf4f8fc94aae3c.jpg
dataset_roboflow/new_train/image_120_png.rf.fd2769205414dbeac9c28945bd26f99e.jpg
dataset_roboflow/new_train/images_12_png.rf.387ec5af908dcae46e520a8ef6648508.jpg
dataset_roboflow/new_train/image_93_png.rf.5ec8d17d5e27f6d6f

dataset_roboflow/new_train/image_199_png.rf.4ad9745c6688cf1585a112f4e263d2d4.jpg
dataset_roboflow/valid/image_83_png.rf.6cc9b30a61534d606877f35883890efc.jpg
dataset_roboflow/new_train/image_86_png.rf.5e367fcb50cd83e170a8b815efc7c228.jpg
dataset_roboflow/new_train/00056_png.rf.8184ad18f95958f0e431d08b32813fbe.jpg
dataset_roboflow/valid/image_6_png.rf.dbdfb20ff6104793a5569be0c7fc2901.jpg
dataset_roboflow/train/images_2_png.rf.ae386904d0d9eeeeb4c431f1c23b10e5.jpg
dataset_roboflow/train/image_184_png.rf.080daf927fb5dd6c4002f98d5b606a16.jpg
dataset_roboflow/train/image_37_png.rf.2001162d1409b2f73fcf3aac50eb4cdc.jpg
dataset_roboflow/train/image_101_png.rf.29208c72003a4b32f15ea62b15af921f.jpg
dataset_roboflow/label_preprocess_for_trainingSet_and_valSet.ipynb
dataset_roboflow/new_train/image_56_png.rf.2d71e0301a22117eabeaaef7cf40ec7a.jpg
dataset_roboflow/train/image_161_png.rf.cfd20b4fe1695e91f26646da22150381.jpg
dataset_roboflow/test/image_131_png.rf.30cd35691e523cf75b6cff4b23bc57fe.jpg
datas

dataset_roboflow/new_train/images_1_png.rf.277eaf560622b762b34e3f3325ecfccb.jpg
dataset_roboflow/new_train/image_157_png.rf.25f4ca2d300bf8ba063bd65a0184d341.jpg
dataset_roboflow/test/images_14_png.rf.2e1996b917db3d4b700281e4223384a1.jpg
dataset_roboflow/train/image_1_png.rf.5d188e903b4d976800a1808a02479bae.jpg
dataset_roboflow/train/00016_png.rf.c01c97143ac974f20977499cba2436a0.jpg
dataset_roboflow/new_train/image_114_png.rf.e1e6c69eef7dc0d347ea14e9f69f1a3d.jpg
dataset_roboflow/train/image_64_png.rf.14b6cf9c5a4e392557ce86385a758df3.jpg
dataset_roboflow/new_train/image_126_png.rf.5cd69fb531a55617d7485cc1e7f9bbb8.jpg
dataset_roboflow/train/image_52_png.rf.2bf5b466fe6df6a4f38d59261d0aa826.jpg
dataset_roboflow/train/image_58_png.rf.8bd99697bf3ffee215330003b90d7399.jpg
dataset_roboflow/new_train/image_170_png.rf.f2f08ad86240a4ef89d3d6846df71569.jpg
dataset_roboflow/new_train/image_164_png.rf.8b7d30591ae97dbe147928429a8fbd78.jpg
dataset_roboflow/new_train/00037_png.rf.15c3470189819096fe8a96e

dataset_sketch_it/images/ce3baaad-b073-4694-8ae0-1cb1e818c075.png
dataset_sketch_it/images/0e883baf-e153-4a42-8e0c-f8cde7c179c3.png
dataset_sketch_it/images/9e699db6-adf6-42ea-a833-1a4e5633f029.png
dataset_sketch_it/images/778ff65f-b1de-4f58-b2ca-cf9fb3d02e96.png
dataset_sketch_it/images/87fb3dd7-8551-41f3-bcb1-da87ddeccf6e.png
dataset_sketch_it/images/261d5124-d34b-49e2-89bf-105028836be8.png
dataset_sketch_it/images/808d5476-08eb-4b03-8997-6d16d1e2a32b.png
dataset_sketch_it/images/4e16d9d6-be4b-46f5-99e1-8d144b11ecde.png
dataset_sketch_it/images/80ce0ae6-9f89-429b-b2da-ae1b25e45acb.png
dataset_sketch_it/images/bd73cef8-a70e-483e-b0f5-81a7a81ca95d.png
dataset_sketch_it/images/722b8c8f-7975-4fc7-abde-b27b3bd37883.png
dataset_sketch_it/images/6bec6664-a131-4e83-ae7d-afa331892d7c.png
dataset_sketch_it/images/9bf34aeb-8b91-4061-b67e-cc08a3c7d177.png
dataset_sketch_it/images/52840896-0073-4c1f-9aca-bbd87c17eac9.png
dataset_sketch_it/images/8cea0797-e148-4eff-be0a-08bd8dad494a.png
dataset_sk

In [2]:
new_dataset_path='./new_dataset/'


In [3]:
import numpy as np
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import pandas as pd
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from pandas.plotting import register_matplotlib_converters

import urllib
import os
import csv
import cv2
import time
from PIL import Image

from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color



# from keras_retinanet import models/
# from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
# from keras_retinanet.utils.visualization import draw_box, draw_caption
# from keras_retinanet.utils.colors import label_color

%matplotlib inline
%config InlineBackend.figure_format='retina'

register_matplotlib_converters()


RANDOM_SEED = 42

np.random.seed(RANDOM_SEED)


In [4]:
THRES_SCORE = 0.4

def draw_detections(image, box, scores=0, label=''):
#   for box, score, label in zip(boxes[0], scores[0], labels[0]):
#     if score < THRES_SCORE:
#         continue

#     color = label_color(labels)

#     box = box.astype(int)
    
    draw_box(image, box, color=(0,255,0) )

    caption = "{} {:.3f}".format(label, 0)
    draw_caption(image, box, caption)
    
    
    

def show_detected_objects(image_row,img_folder=''):
    
    img_path = img_folder+image_row.image_name
    true_box = [
        image_row.x_min, image_row.y_min, image_row.x_max, image_row.y_max]
    image = read_image_bgr(img_path)

    draw = image.copy()
    draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

    image = preprocess_image(image)
    image, scale = resize_image(image)

#     boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis = 0))
#     print('scale',scale)
#     print('scores',scores)
#     print('labels',labels)
#     print('image',image)

#     boxes /= scale
#     print('boxes',boxes[0])

    draw_box(draw, true_box, color=(255, 0, 0))
    draw_detections(image, box=true_box, label= labels_to_name)
    
#     caption = "{} {:.3f}".format(image_row.class_name,0)
#     draw_caption(image,true_box , image_row.class_name)

    plt.axis('off')
    plt.imshow(draw)
    plt.show()

In [5]:
roboflow = pd.read_csv('dataset_roboflow/new_train/annotations.csv')
sketch_it = pd.read_csv('dataset_sketch_it/images/annotations.csv')

In [6]:
roboflow

,image_name,width,height,class_name,x_min,y_min,x_max,y_max
0,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,95,98,345,157
1,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,102,244,342,309
2,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Button,106,542,372,614
3,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,12,160,139,400
4,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,191,151,306,395
...,...,...,...,...,...,...,...,...
4577,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,44,288,621,415
4578,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,27,157,257,275
4579,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,277,153,620,269
4580,images_14_png.rf.2e1996b917db3d4b700281e422338...,640,640,Carousel,319,34,607,133


In [7]:
sketch_it

,image_name,width,height,class_name,x_min,y_min,x_max,y_max
0,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,423,332,690,414
1,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Button,989,422,1186,554
2,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,Label,18,146,328,207
3,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,CheckBox,989,220,1292,294
4,2469da39-65f0-428b-a1de-cedbba7db32e.png,1600,564,RadioButton,804,107,952,168
...,...,...,...,...,...,...,...,...
1996,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,453,628,645,678
1997,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,133,734,369,855
1998,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,300,204,1072,478
1999,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,752,639,983,679


In [8]:
new_dataset = roboflow.append(sketch_it,ignore_index=True)
new_dataset

/tmp/ipykernel_29667/2169078318.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  new_dataset = roboflow.append(sketch_it,ignore_index=True)


,image_name,width,height,class_name,x_min,y_min,x_max,y_max
0,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,95,98,345,157
1,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,102,244,342,309
2,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Button,106,542,372,614
3,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,12,160,139,400
4,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,191,151,306,395
...,...,...,...,...,...,...,...,...
6578,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,453,628,645,678
6579,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,133,734,369,855
6580,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,300,204,1072,478
6581,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,752,639,983,679


In [9]:
new_dataset.rename(columns={'image_name': 'filename'}, inplace=True)
new_dataset.rename(columns={'x_min': 'xmin'}, inplace=True)
new_dataset.rename(columns={'y_min': 'ymin'}, inplace=True)
new_dataset.rename(columns={'x_max': 'xmax'}, inplace=True)
new_dataset.rename(columns={'y_max': 'ymax'}, inplace=True)
new_dataset.rename(columns={'class_name': 'label'}, inplace=True)


roboflow.rename(columns={'image_name': 'filename'}, inplace=True)
roboflow.rename(columns={'x_min': 'xmin'}, inplace=True)
roboflow.rename(columns={'y_min': 'ymin'}, inplace=True)
roboflow.rename(columns={'x_max': 'xmax'}, inplace=True)
roboflow.rename(columns={'y_max': 'ymax'}, inplace=True)
roboflow.rename(columns={'class_name': 'label'}, inplace=True)

sketch_it.rename(columns={'image_name': 'filename'}, inplace=True)
sketch_it.rename(columns={'x_min': 'xmin'}, inplace=True)
sketch_it.rename(columns={'y_min': 'ymin'}, inplace=True)
sketch_it.rename(columns={'x_max': 'xmax'}, inplace=True)
sketch_it.rename(columns={'y_max': 'ymax'}, inplace=True)
sketch_it.rename(columns={'class_name': 'label'}, inplace=True)

new_dataset

,filename,width,height,label,xmin,ymin,xmax,ymax
0,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,95,98,345,157
1,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,102,244,342,309
2,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Button,106,542,372,614
3,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,12,160,139,400
4,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,191,151,306,395
...,...,...,...,...,...,...,...,...
6578,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,453,628,645,678
6579,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,133,734,369,855
6580,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,300,204,1072,478
6581,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,752,639,983,679


In [10]:
import shutil
from PIL import Image
import cv2
import imghdr

def copy2dri(df,destination,fromDir=[]):

    print(destination)

    for i in df.filename: 


    #     create dir
        if not os.path.exists(destination):
            os.makedirs(destination)

        for Dir in fromDir:
#             print(f'{Dir+i} -')
            
            if os.path.exists(Dir+i):
                pli_img = Image.open(Dir+i)  
                cv_img = cv2.imread(Dir+i)

#                 print(f'{Dir+i} - {pli_img.format}')
        #         cv2.imwrite(destination+'/'+i, cv_img, [int(cv2.IMWRITE_JPEG_QUALITY), 100])  
                shutil.copy2(Dir+i,destination)


    df.to_csv(destination+'/annotations.csv',index=False)      

    return 'done'

In [11]:

copy2dri(destination=new_dataset_path,df= new_dataset,fromDir=['dataset_roboflow/new_train/','dataset_sketch_it/images/'])

# destination='./new_dataset'
# 'dataset_roboflow/new_train/'
# 'dataset_sketch_it/images/'

./new_dataset/


'done'

In [12]:
new_dataset = pd.read_csv('./new_dataset/annotations.csv')
new_dataset

,filename,width,height,label,xmin,ymin,xmax,ymax
0,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,95,98,345,157
1,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,102,244,342,309
2,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Button,106,542,372,614
3,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,12,160,139,400
4,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,191,151,306,395
...,...,...,...,...,...,...,...,...
6578,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,453,628,645,678
6579,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,133,734,369,855
6580,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,300,204,1072,478
6581,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,752,639,983,679


# filter by accpted classes

In [ ]:
labels_to_name=pd.DataFrame( new_dataset['class'].unique()).sort_values(by=0)
labels_to_name

In [14]:
new_dataset.label.value_counts()

Label          1075
TextBox         719
CheckBox        695
Button          643
Image           592
Link            491
Heading         490
RadioButton     416
Paragraph       360
Select          351
Table           214
Carousel        178
Textarea        151
Pagination      150
ComboBox         58
Name: label, dtype: int64

In [15]:
accpted_classes = pd.read_csv('./accpted_classes.csv')
accpted_classes

,class_label
0,Button
1,CheckBox
2,Heading
3,Image
4,Label
5,Link
6,Paragraph
7,RadioButton
8,Select
9,TextBox


In [16]:
accpted_classes_list = accpted_classes.values.reshape(1,-1)[0]

classes = []

for i,item in enumerate(accpted_classes_list):
        classes.append({'name':item, 'id':i+1})


with open('label_map.pbtxt', 'w') as f:
    for item in classes:
        f.write('item { \n')
        f.write('\tname:\'{}\'\n'.format(item['name']))
        f.write('\tid:{}\n'.format(item['id']))
        f.write('}\n') 

In [18]:
new_dataset = new_dataset[new_dataset.label.isin(list(accpted_classes.values.reshape(1,-1)[0]))]
new_dataset
# accpted_classes.values.reshape(1,-1)[0]

,filename,width,height,label,xmin,ymin,xmax,ymax
0,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,95,98,345,157
1,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Select,102,244,342,309
2,00024_png.rf.097dd8beb3a90a32588e3b7d14105214.jpg,640,640,Button,106,542,372,614
3,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,12,160,139,400
4,00027_png.rf.051c242efd7c63dbc96a34fdfc9e8499.jpg,640,640,Image,191,151,306,395
...,...,...,...,...,...,...,...,...
6578,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,453,628,645,678
6579,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,133,734,369,855
6580,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,300,204,1072,478
6581,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,752,639,983,679


# show all object

In [ ]:



for i in range(3):
    item = new_dataset.iloc[i]
    print(f'{i}- {item.filename} - {item.class}')
    show_detected_objects(item,img_folder='new_dataset/')

# spilt into train and test

In [ ]:
from sklearn.model_selection import train_test_split
import shutil


if os.path.exists('train'):
    shutil.rmtree('train')
    
if os.path.exists('test'):
    shutil.rmtree('test')

train_set, test_set = train_test_split(new_dataset, test_size = 0.2,shuffle=False)


In [ ]:
train_set

In [ ]:
test_set

In [ ]:
copy2dri(destination='train/',df= train_set,fromDir=[new_dataset_path])


In [ ]:
copy2dri(destination='test/',df= test_set,fromDir=[new_dataset_path])


# create coco from csv

In [20]:
import numpy as np
import json
import pandas as pd

path = 'tensorflow2csv.csv'
save_json_path = 'traincoco.json'


data = new_dataset.copy()

images = []
categories = []
annotations = []

category = {}
category["supercategory"] = 'none'
category["id"] = 0
category["name"] = 'None'
categories.append(category)

data['fileid'] = data['filename'].astype('category').cat.codes
data['categoryid']= pd.Categorical(data['label'],ordered= True).codes
data['categoryid'] = data['categoryid']+1
data['annid'] = data.index

def image(row):
    image = {}
    image["height"] = row.height
    image["width"] = row.width
    image["id"] = row.fileid
    image["file_name"] = row.filename
    return image

def category(row):
    category = {}
    category["supercategory"] = 'None'
    category["id"] = row.categoryid
    category["name"] = row.label
    return category

def annotation(row):
    annotation = {}
    area = (row.xmax -row.xmin)*(row.ymax - row.ymin)
    annotation["segmentation"] = []
    annotation["iscrowd"] = 0
    annotation["area"] = area
    annotation["image_id"] = row.fileid

    annotation["bbox"] = [row.xmin, row.ymin, row.xmax -row.xmin,row.ymax-row.ymin ]

    annotation["category_id"] = row.categoryid
    annotation["id"] = row.annid
    return annotation

for row in data.itertuples():
    annotations.append(annotation(row))

imagedf = data.drop_duplicates(subset=['fileid']).sort_values(by='fileid')
for row in imagedf.itertuples():
    images.append(image(row))

catdf = data.drop_duplicates(subset=['categoryid']).sort_values(by='categoryid')
for row in catdf.itertuples():
    categories.append(category(row))

data_coco = {}
data_coco["images"] = images
data_coco["categories"] = categories
data_coco["annotations"] = annotations


json.dump(data_coco, open(save_json_path, "w"), indent=4)

In [41]:
data_coco

{'images': [{'height': 640,
   'width': 640,
   'id': 0,
   'file_name': '00004_png.rf.ca684ae59d6f64243191968d04714a9f.jpg'},
  {'height': 640,
   'width': 640,
   'id': 1,
   'file_name': '00005_png.rf.59ae759d32184862d21e8e47699a3f75.jpg'},
  {'height': 640,
   'width': 640,
   'id': 2,
   'file_name': '00006_png.rf.45899b16808a011685fd656dfa94a2cd.jpg'},
  {'height': 640,
   'width': 640,
   'id': 3,
   'file_name': '00007_png.rf.81264a2f50265f4f2bbe9e0982dec718.jpg'},
  {'height': 640,
   'width': 640,
   'id': 4,
   'file_name': '00008_png.rf.fd4a12657377d42c46ae2a91ab9b770c.jpg'},
  {'height': 640,
   'width': 640,
   'id': 5,
   'file_name': '00009_png.rf.012ebaf23980e28cf41b1be064309889.jpg'},
  {'height': 640,
   'width': 640,
   'id': 6,
   'file_name': '00010_png.rf.ed049aa67c8121919d506c94ebb46711.jpg'},
  {'height': 640,
   'width': 640,
   'id': 7,
   'file_name': '00011_png.rf.16f88786c470a59efda7558d410d682e.jpg'},
  {'height': 640,
   'width': 640,
   'id': 8,
   'fil

In [40]:
data[data.filename=='39f28af1-a6d1-4620-9d9d-840450a1e6a2.png']

,filename,width,height,label,xmin,ymin,xmax,ymax,fileid,categoryid,annid
6569,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Image,734,475,896,623,61,4,6569
6570,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Image,421,478,600,623,61,4,6570
6571,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Paragraph,756,738,984,863,61,7,6571
6572,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Label,462,687,666,730,61,5,6572
6573,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Label,756,680,995,740,61,5,6573
6574,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Label,148,688,361,739,61,5,6574
6575,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,397,31,814,175,61,3,6575
6576,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,160,638,349,686,61,3,6576
6577,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Image,132,455,338,628,61,4,6577
6578,39f28af1-a6d1-4620-9d9d-840450a1e6a2.png,1200,900,Heading,453,628,645,678,61,3,6578


In [34]:
len(new_dataset['filename'].value_counts())

314

In [32]:
pd.Categorical(data['label'],ordered= True).codes

array([8, 8, 0, ..., 6, 2, 6], dtype=int8)

In [35]:
data['filename'].astype('category').cat.codes

0       20
1       20
2       20
3       23
4       23
        ..
6578    61
6579    61
6580    61
6581    61
6582    61
Length: 5832, dtype: int16

# create tf record

In [ ]:

from __future__ import division
from __future__ import print_function
from __future__ import absolute_import

import os
import io
import sys
import pandas as pd
import tensorflow.compat.v1 as tf
# import tensorflow as tf

from PIL import Image
from object_detection.utils import dataset_util, label_map_util
from collections import namedtuple, OrderedDict

# flags = tf.app.flags
# flags.DEFINE_string('csv_input', '', 'Path to the CSV input')
# flags.DEFINE_string('output_path', '', 'Path to output TFRecord')
# flags.DEFINE_string('image_dir', '', 'Path to images')
# FLAGS = flags.FLAGS

class_map = label_map_util.load_labelmap('label_map.pbtxt')
class_map_dict = label_map_util.get_label_map_dict(class_map)

# print(class_map_dict)



# TO-DO replace this with label map
def class_text_to_int(row_label):
    return class_map_dict[row_label]


def split(df, group):
    data = namedtuple('data', ['image_name', 'object'])
    gb = df.groupby(group)
    return [data(image_name, gb.get_group(x)) for image_name, x in zip(gb.groups.keys(), gb.groups)]


def create_tf_example(group, path):
    
    with tf.gfile.GFile(os.path.join(path, '{}'.format(group.image_name)), 'rb') as fid:
        encoded_jpg = fid.read()
    encoded_jpg_io = io.BytesIO(encoded_jpg)
    image = Image.open(encoded_jpg_io)

    width, height = image.size

#     print(encoded_jpg)
    
    image_name = group.image_name.encode('utf8')
#     print(image_name)
    image_format = image.format.encode()
    xmins = []
    xmaxs = []
    ymins = []
    ymaxs = []
    classes_text = []
    classes_num = []

    for index, row in group.object.iterrows():
#         print(row['x_min'])
        xmins.append(row['x_min'] / width)
        xmaxs.append(row['x_max'] / width)
        ymins.append(row['y_min'] / height)
        ymaxs.append(row['y_max'] / height)
        classes_text.append(row['class_name'].encode('utf8'))
        classes_num.append(class_text_to_int(row['class_name']))
        
#     print(image_format)


    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': dataset_util.int64_feature(height),
        'image/width': dataset_util.int64_feature(width),
        'image/filename': dataset_util.bytes_feature(image_name),
        'image/source_id': dataset_util.bytes_feature(image_name),
        'image/encoded': dataset_util.bytes_feature(encoded_jpg),
        'image/format': dataset_util.bytes_feature(image_format),
        'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
        'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
        'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
        'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
        'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
        'image/object/class/label': dataset_util.int64_list_feature(classes_num),
    }))
        
    return tf_example


def csv_2_tfrecord(output_path,image_dir,csv_input):
    
    writer = tf.python_io.TFRecordWriter(output_path)
    path = os.path.join(image_dir)
    examples = pd.read_csv(csv_input)
    grouped = split(examples, 'image_name')
#     print(grouped)
    

    # added
    file_errors = 0

    for group in grouped:
        try:
            tf_example = create_tf_example(group, path)
            writer.write(tf_example.SerializeToString())
        except:

            # added
            file_errors += 1
            pass

    writer.close()

    # added
    print("FINISHED. There were %d errors" % file_errors)

    output_path = os.path.join(os.getcwd(), output_path)
    print('Successfully created the TFRecords: {}'.format(output_path))

    
# call
csv_2_tfrecord(csv_input='train/annotations.csv',
              image_dir='train/',
              output_path= 'train.record')

csv_2_tfrecord(csv_input='test/annotations.csv',
              image_dir='test/',
              output_path= 'test.record')





In [ ]:
classes